# Pediatric Pneumonia Detection using InceptionV3

## Overview
This notebook implements transfer learning with InceptionV3 for automated detection and classification of pneumonia from pediatric chest X-rays.

### Model Architecture
- Base: InceptionV3 pretrained on ImageNet
- Modified layers: Mixed_7b, Mixed_7c, and final FC
- Output classes: 3 (Normal, Bacterial Pneumonia, Viral Pneumonia)

### Training Configuration
- Learning rate: 0.0001
- Batch size: 32
- Early stopping patience: 3
- Data augmentation: light rotation, translation, scaling

### Environment Requirements
- Python 3.10
- PyTorch 2.1.0
- torchvision 0.16.0
- CUDA compatible GPU

### Dataset
Uses the Chest X-Ray Images (Pneumonia) dataset containing:
- 5,856 validated chest X-ray images
- Age range: 1-5 years
- Categories: Normal, Bacterial Pneumonia, Viral Pneumonia

## 1. Environment Setup

Mount Google Drive for accessing the dataset and saving model weights.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Copy training and testing data from Google Drive to local environment.

In [ ]:
!cp -r /content/drive/MyDrive/datasets/train /content/train
!cp -r /content/drive/MyDrive/datasets/test /content/test

## 2. Data Exploration

Analyze dataset distribution and create training/validation split.

In [ ]:
import os

# Define the paths to train and test directories
train_dir = '/content/train'
test_dir = '/content/test'

# Count number of examples per class in the training set
for class_dir in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_dir)
    if os.path.isdir(class_path):
        num_examples = len(os.listdir(class_path))
        print(f'Number of examples in {class_dir}: {num_examples}')

# Count number of examples per class in the test set
for class_dir in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_dir)
    if os.path.isdir(class_path):
        num_examples = len(os.listdir(class_path))
        print(f'Number of examples in {class_dir}: {num_examples}')

Number of examples in viral pneumonia: 1345
Number of examples in bacterial pneumonia: 2530
Number of examples in a_normal_xrays: 1341
Number of examples in viral pneumonia: 148
Number of examples in bacterial pneumonia: 242
Number of examples in a_normal_xrays: 234


Create training/validation split (90/10 percent).

In [ ]:
#Create Validation Set using 10% of train set
from sklearn.model_selection import train_test_split

# Prepare empty lists to hold file paths and labels
file_paths = []
labels = []

# Assign integer labels for each class (assuming three classes)
class_map = {'viral pneumonia': 0, 'bacterial pneumonia': 1, 'a_normal_xrays': 2}

# Iterate through each class folder and gather file paths and labels
for class_name, class_label in class_map.items():
    class_folder = os.path.join(train_dir, class_name)
    for file_name in os.listdir(class_folder):
        file_path = os.path.join(class_folder, file_name)
        file_paths.append(file_path)
        labels.append(class_label)

# Split the data (10% for validation, 90% for training)
X_train, X_val, y_train, y_val = train_test_split(
    file_paths, labels, test_size=0.1, random_state=42, stratify=labels)

# Check the number of images
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")

Training set size: 4694
Validation set size: 522


In [ ]:
import shutil

# Define the path to your datasets folder in Google Drive
validation_folder = '/content/drive/MyDrive/datasets/validation'

# Create subdirectories for each class in the validation folder
for class_name in class_map.keys():
    os.makedirs(os.path.join(validation_folder, class_name), exist_ok=True)

# Move validation files to the corresponding class folder in the validation folder
for file_path, label in zip(X_val, y_val):
    # Get the class name corresponding to the label
    class_name = list(class_map.keys())[list(class_map.values()).index(label)]

    # Define the destination path in the validation folder
    destination_path = os.path.join(validation_folder, class_name, os.path.basename(file_path))

    # Copy the file to the validation folder in Google Drive
    shutil.copy(file_path, destination_path)

print("Validation set copied to Google Drive.")

Validation set copied to Google Drive.


In [ ]:
!cp -r /content/drive/MyDrive/datasets/validation /content/validation

## 3. Data Preprocessing and Loading

Calculate dataset statistics for normalization and prepare data transforms.

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Initial transform: Resize and convert to tensor only
simple_transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Ensure all images are the same size
    transforms.ToTensor()  # Convert images to tensor format
])

# Load dataset with simple transform (no normalization)
train_dataset_simple = datasets.ImageFolder(root='/content/train', transform=simple_transform)

# Create a DataLoader for the training data
train_loader_simple = DataLoader(train_dataset_simple, batch_size=32, shuffle=False, num_workers=2)

def compute_mean_std(loader):
    mean = 0.0
    std = 0.0
    total_images_count = 0
    for images, _ in loader:
        batch_samples = images.size(0)  # Number of images in the batch
        images = images.view(batch_samples, images.size(1), -1)  # Flatten the images
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count
    return mean, std

# Calculate mean and standard deviation
mean, std = compute_mean_std(train_loader_simple)

print(f"Mean: {mean}, Std: {std}")

Mean: tensor([0.4823, 0.4823, 0.4823]), Std: tensor([0.2220, 0.2220, 0.2220])


Load and Normalize datasets

In [ ]:
# Use the computed mean and std values and perform data augmentation on train dataset
train_transform = transforms.Compose([
       transforms.RandomRotation(10),  # Small rotations for generalization
       transforms.RandomAffine(degrees=0, translate=(0.03, 0.03)),  # Small translations
       transforms.RandomResizedCrop(299, scale=(0.95, 1.05)),  # Slight zoom
       transforms.ToTensor(),  # Convert to tensor
       transforms.Normalize(mean=mean.tolist(), std=std.tolist())  # Use computed mean and std
   ])

#Only resize and normalize test dataset
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())  # Same normalization for test set
])


# Load dataset (assumes dataset is in a directory with subfolders for each class)
train_dataset = datasets.ImageFolder(root='/content/train', transform=train_transform)
val_dataset = datasets.ImageFolder(root='/content/validation', transform=test_transform)
test_dataset = datasets.ImageFolder(root='/content/test', transform=test_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers= 2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class to index mapping
print(train_dataset.class_to_idx)  # {'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}
print(val_dataset.class_to_idx)  # {'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}
print(test_dataset.class_to_idx)  # {'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}

{'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}
{'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}
{'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}


## 4. Model Architecture

Initialize InceptionV3 model with transfer learning setup.

In [ ]:
import torch.nn as nn
import torchvision.models as models

# Load  pretrained InceptionV3 model
model = models.inception_v3(pretrained=True)

# Freeze the pre-trained layers (except the last fully connected layer)
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer for 3 classes (pneumonia types)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 3 classes: normal, bacterial pneumonia, viral pneumonia

# Ensure the final layer parameters are trainable
for param in model.fc.parameters():
    param.requires_grad = True

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 137MB/s] 


## 5. Training Configuration

Configure training parameters:
- Loss function: Cross Entropy
- Optimizer: Adam
- Learning rate scheduling (starting with .0001)
- Early stopping

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

# Set up learning rate scheduler with patience = 2
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Early stopping parameters
patience = 3  # Stop training if no improvement after 3 epochs
early_stopping_counter = 0
best_val_loss = float('inf')

# Training loop
num_epochs = 20  # Adjust the number of epochs

for epoch in range(num_epochs):
    # Print the current learning rate
    for param_group in optimizer.param_groups:
        print(f"Epoch [{epoch+1}/{num_epochs}] - Current Learning Rate: {param_group['lr']}")

    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training phase
    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.logits, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    # Print training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            # Move images and labels to the device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    # Calculate validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Check for learning rate scheduling
    scheduler.step(val_loss)

    # Early stopping based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0  # Reset the early stopping counter
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

print('Training finished.')

# Define the path where you want to save the model weights in Google Drive
save_path = '/content/drive/MyDrive/datasets/Inception_1layer_nodropout_weights.pth'

# After training is finished, save the model weights
torch.save(model.state_dict(), save_path)

print(f'Model weights saved to {save_path}')

Epoch [1/20] - Current Learning Rate: 0.0001
Epoch [1/20], Loss: 0.7768, Training Accuracy: 68.25%
Validation Loss: 0.8052, Validation Accuracy: 69.92%
Epoch [2/20] - Current Learning Rate: 0.0001
Epoch [2/20], Loss: 0.7284, Training Accuracy: 71.03%
Validation Loss: 0.7705, Validation Accuracy: 71.46%
Epoch [3/20] - Current Learning Rate: 0.0001
Epoch [3/20], Loss: 0.7024, Training Accuracy: 71.91%
Validation Loss: 0.7537, Validation Accuracy: 72.03%
Epoch [4/20] - Current Learning Rate: 0.0001
Epoch [4/20], Loss: 0.6943, Training Accuracy: 72.35%
Validation Loss: 0.7516, Validation Accuracy: 71.46%
Epoch [5/20] - Current Learning Rate: 0.0001
Epoch [5/20], Loss: 0.6708, Training Accuracy: 72.51%
Validation Loss: 0.7210, Validation Accuracy: 72.61%
Epoch [6/20] - Current Learning Rate: 0.0001
Epoch [6/20], Loss: 0.6560, Training Accuracy: 73.68%
Validation Loss: 0.7248, Validation Accuracy: 71.84%
Epoch [7/20] - Current Learning Rate: 0.0001
Epoch [7/20], Loss: 0.6496, Training Accura

## 6. Additional Training

Configure training parameters:
- Loss function: Cross Entropy
- Optimizer: Adam
- Learning rate scheduling
- Early stopping
- Monitor training and validation accuracy to watch for overfitting

In [ ]:
# Set up loss function and optimizer (you don't need to change this part)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

# Set up learning rate scheduler with patience = 2
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Early stopping parameters
patience = 3  # Stop training if no improvement after 3 epochs
early_stopping_counter = 0
best_val_loss = float('inf')

# Continue Training Loop for additional epochs
additional_epochs = 20  # Set how many additional epochs you want to train for

for epoch in range(additional_epochs):
    # Print the current learning rate
    for param_group in optimizer.param_groups:
        print(f"Epoch [{epoch+1}/{additional_epochs}] - Current Learning Rate: {param_group['lr']}")

    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training phase
    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.logits, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    # Print training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch [{epoch+1}/{additional_epochs}], Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            # Move images and labels to the device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    # Calculate validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Check for learning rate scheduling
    scheduler.step(val_loss)

    # Early stopping based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0  # Reset the early stopping counter
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

print('Training finished.')

# Save the model weights after additional training
save_path = '/content/drive/MyDrive/datasets/Inception_1layer_nodropout_weights_continued.pth'
torch.save(model.state_dict(), save_path)
print(f'Model weights saved to {save_path}')

Epoch [1/20] - Current Learning Rate: 0.0001
Epoch [1/20], Loss: 0.5966, Training Accuracy: 75.13%
Validation Loss: 0.6684, Validation Accuracy: 73.56%
Epoch [2/20] - Current Learning Rate: 0.0001
Epoch [2/20], Loss: 0.6001, Training Accuracy: 75.29%
Validation Loss: 0.6793, Validation Accuracy: 74.14%
Epoch [3/20] - Current Learning Rate: 0.0001
Epoch [3/20], Loss: 0.5809, Training Accuracy: 75.88%
Validation Loss: 0.6549, Validation Accuracy: 74.14%
Epoch [4/20] - Current Learning Rate: 0.0001
Epoch [4/20], Loss: 0.5912, Training Accuracy: 74.90%
Validation Loss: 0.6532, Validation Accuracy: 74.71%
Epoch [5/20] - Current Learning Rate: 0.0001
Epoch [5/20], Loss: 0.5827, Training Accuracy: 75.98%
Validation Loss: 0.6489, Validation Accuracy: 74.33%
Epoch [6/20] - Current Learning Rate: 0.0001
Epoch [6/20], Loss: 0.5830, Training Accuracy: 75.54%
Validation Loss: 0.6443, Validation Accuracy: 74.71%
Epoch [7/20] - Current Learning Rate: 0.0001
Epoch [7/20], Loss: 0.5904, Training Accura

In [ ]:
import torch.nn.functional as F

# Function to evaluate the model on the test dataset
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode (no gradient computation)
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in test_loader:
            # Move images and labels to the device (GPU)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Example usage after training
evaluate_model(model, test_loader)

Accuracy: 80.13%


## 7. Update Training Configuration and Continue
- Unfreeze another layer and continue training for 20 more epochs

In [ ]:
# Unfreeze the last two Inception blocks (Mixed_7b and Mixed_7c)
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers first

# Unfreeze the last two inception blocks and the fully connected layer
for name, param in model.named_parameters():
    if 'Mixed_7b' in name or 'Mixed_7c' in name or 'fc' in name:
        param.requires_grad = True  # Unfreeze the specified layers

# Move the model to the appropriate device
model = model.to(device)


In [ ]:
# Update the optimizer to include the trainable parameters
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

# Training loop
num_epochs = 20  # Adjust the number of epochs

for epoch in range(num_epochs):
    # Print the current learning rate
    for param_group in optimizer.param_groups:
        print(f"Epoch [{epoch+1}/{num_epochs}] - Current Learning Rate: {param_group['lr']}")

    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training phase
    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.logits, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    # Print training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            # Move images and labels to the device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    # Calculate validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Check for learning rate scheduling
    scheduler.step(val_loss)

    # Early stopping based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0  # Reset the early stopping counter
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

print('Training finished.')

# Define the path where you want to save the model weights in Google Drive
save_path = '/content/drive/MyDrive/datasets/Inception_1layer_nodropout_weights_3.pth'

# After training is finished, save the model weights
torch.save(model.state_dict(), save_path)

print(f'Model weights saved to {save_path}')


Epoch [1/20] - Current Learning Rate: 0.0001
Epoch [1/20], Loss: 0.5169, Training Accuracy: 77.80%
Validation Loss: 0.4560, Validation Accuracy: 82.18%
Epoch [2/20] - Current Learning Rate: 0.0001
Epoch [2/20], Loss: 0.4261, Training Accuracy: 81.54%
Validation Loss: 0.4034, Validation Accuracy: 82.57%
Epoch [3/20] - Current Learning Rate: 0.0001
Epoch [3/20], Loss: 0.3622, Training Accuracy: 84.43%
Validation Loss: 0.3588, Validation Accuracy: 85.82%
Epoch [4/20] - Current Learning Rate: 0.0001
Epoch [4/20], Loss: 0.3247, Training Accuracy: 85.99%
Validation Loss: 0.3289, Validation Accuracy: 88.89%
Epoch [5/20] - Current Learning Rate: 0.0001
Epoch [5/20], Loss: 0.2704, Training Accuracy: 88.57%
Validation Loss: 0.3032, Validation Accuracy: 88.70%
Epoch [6/20] - Current Learning Rate: 0.0001
Epoch [6/20], Loss: 0.2303, Training Accuracy: 90.38%
Validation Loss: 0.3372, Validation Accuracy: 89.08%
Epoch [7/20] - Current Learning Rate: 0.0001
Epoch [7/20], Loss: 0.2022, Training Accura